# CSV Concat Agent 

## Mission

Create TWO versions of a CSV concatenation function and experiment to see which one the agent can successfully call.

**Version 1: Parameterized** - Takes folder_path and output_file as parameters  
**Version 2: Hardcoded** - Has paths hardcoded inside

Then try different:
- ✅ Questions (5+ variations)
- ✅ System prompts (3+ variations)
- ✅ Docstrings (2+ variations)

**Goal:** Understand how agents interact with functions and what makes them actually execute vs just describe!

---
## Setup

In [ ]:
import sys
!{sys.executable} -m pip install llama-index llama-index-llms-ollama


In [1]:
import pandas as pd
import os
from typing import Dict
import requests

from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.tools import FunctionTool
from llama_index.llms.ollama import Ollama

print("✅ Imports successful!")

✅ Imports successful!


In [2]:
# Get available model
LLM_BASE_URL = "http://localhost:11434"

models = requests.get(f'{LLM_BASE_URL}/api/tags').json().get('models', [])

if models:
    llm_model = models[0].get('name', 'NA')
else:
    llm_model = 'NA'

print(f'🏷️ Using model: {llm_model}')

🏷️ Using model: llama3.2:1b


---
## Part 1: Create Two Function Versions

### Version 1: Parameterized Function

**TODO:** Create a function that takes `folder_path` and `output_file` as parameters.

**Tips:**
- Write a CLEAR docstring (the agent reads this!)
- Explain what each parameter does
- Give an example in the docstring

In [3]:
def concat_csv_files(folder_path: str, output_file: str) -> Dict:
    """
    Concatenate all CSV files in a given folder into a single CSV file.

    This function scans the specified folder for `.csv` files, reads each one
    into a pandas DataFrame, concatenates them row-wise, and saves the combined
    result to the specified output file path.

    Parameters:
        folder_path (str):
            Path to the folder containing CSV files to concatenate.
            Example: "./hogwarts_data"

        output_file (str):
            Path (including filename) where the merged CSV should be saved.
            Example: "./merged_data.csv"

    Returns:
        Dict:
            A dictionary containing:
            - success (bool): Whether the operation succeeded
            - message (str): Status message
            - files_combined (int): Number of CSV files processed
            - total_rows (int): Total number of rows in the merged file
            - output_file (str): Path to the saved CSV file

    Example:
        concat_csv_files(
            folder_path="./hogwarts_data",
            output_file="./merged_data.csv"
        )
    """
    try:
        # TODO: Implement the function
        # 1. Check if folder exists
        if not os.path.isdir(folder_path):
            raise ValueError(f"Folder does not exist: {folder_path}")

        # 2. Get all CSV files from folder
        csv_files = [
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if f.endswith(".csv")
        ]

        if not csv_files:
            raise ValueError("No CSV files found in the specified folder.")

        # 3. Read each CSV into a DataFrame
        dataframes = [pd.read_csv(file) for file in csv_files]

        # 4. Concatenate all DataFrames
        merged_df = pd.concat(dataframes, ignore_index=True)

        # 5. Save to output_file
        merged_df.to_csv(output_file, index=False)

        # 6. Return success dictionary
        return {
            'success': True,
            'message': f"Successfully combined {len(csv_files)} CSV files.",
            'files_combined': len(csv_files),
            'total_rows': len(merged_df),
            'output_file': output_file
        }
        
    except Exception as e:
        return {
            'success': False,
            'message': f"Error: {str(e)}",
            'files_combined': 0,
            'total_rows': 0,
            'output_file': None
        }

print("✓ Parameterized function defined")

✓ Parameterized function defined


### Version 2: Hardcoded Function

**TODO:** Create the SAME function but with hardcoded paths inside.

**Hardcode these values:**
- folder_path = "./hogwarts_data"
- output_file = "./merged_data.csv"

In [4]:
def concat_csv_files_simple() -> Dict:
    """
    Concatenate all CSV files from a predefined folder into a single CSV file.

    This simplified version uses hardcoded paths for both the input folder
    and the output file. The agent does not need to extract or infer any
    parameters — it only needs to call this function.

    Hardcoded paths:
        Input folder: ./hogwarts_data
        Output file: ./merged_data_simple.csv

    Returns:
        Dict:
            A dictionary containing:
            - success (bool): Whether the operation succeeded
            - message (str): Status message
            - files_combined (int): Number of CSV files processed
            - total_rows (int): Total number of rows in the merged file
            - output_file (str): Path to the saved CSV file

    Example:
        concat_csv_files_simple()
    """
    try:
        # TODO: Implement with hardcoded paths
        folder_path = "./hogwarts_data"
        output_file = "./merged_data_simple.csv"
        
        # TODO: Same logic as parameterized version
        # 1. Check if folder exists
        if not os.path.isdir(folder_path):
            raise ValueError(f"Folder does not exist: {folder_path}")

        # 2. Get all CSV files
        csv_files = [
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if f.endswith(".csv")
        ]

        if not csv_files:
            raise ValueError("No CSV files found in the specified folder.")

        # 3. Read and concatenate CSV files
        dataframes = [pd.read_csv(file) for file in csv_files]
        merged_df = pd.concat(dataframes, ignore_index=True)

        # 4. Save to output file
        merged_df.to_csv(output_file, index=False)

        # 5. Return success dictionary
        return {
            'success': True,
            'message': f"Successfully combined {len(csv_files)} CSV files.",
            'files_combined': len(csv_files),
            'total_rows': len(merged_df),
            'output_file': output_file
        }
        
    except Exception as e:
        return {
            'success': False,
            'message': f"Error: {str(e)}",
            'files_combined': 0,
            'total_rows': 0,
            'output_file': None
        }

print("✓ Simple function defined")

✓ Simple function defined


---
## Part 2: Create Agent Tools

Convert both functions to tools that the agent can use.

In [5]:
# Create tool for parameterized version
concat_tool = FunctionTool.from_defaults(
    fn=concat_csv_files,
    name="concat_csv_files",
    description="TODO: Write a clear description. What does this tool do? What parameters does it need?"
)

# Create tool for simple version
concat_simple_tool = FunctionTool.from_defaults(
    fn=concat_csv_files_simple,
    name="concat_csv_files_simple",
    description="TODO: Write description for the simple version."
)

print("✓ Tools created")

✓ Tools created


---
## Part 3: Experimentation

Now the fun part! Try to get the agent to call your functions.

### Experiment 1: Test with Parameterized Function

In [7]:
# Create agent with ONLY the parameterized function
llm = Ollama(model=llm_model, base_url=LLM_BASE_URL, request_timeout=120.0)

agent_param = FunctionAgent(
    tools=[concat_tool],  # Only parameterized version
    llm=llm,
    system_prompt="""

    You are a CSV data assistant.

    Your job is to help users combine CSV files when they ask for it.
    You have access to ONE tool that concatenates CSV files and requires two parameters:
    folder_path and output_file.

    When a user asks to merge/concatenate CSV files:
    1) Identify folder_path and output_file from the user's message.
    2) If both are present, CALL the tool immediately with those exact values.
    3) If either is missing, ask a clarifying question (do not guess).
    Do not describe the function—execute it when possible.

    Example:
    User: "Merge the CSV files in ./hogwarts_data and save as ./merged_data.csv"
    Action: Call the tool with folder_path="./hogwarts_data", output_file="./merged_data.csv"
    """
)

print("✓ Agent created with parameterized function")

✓ Agent created with parameterized function


In [8]:
# Helper function to query agent
async def query_agent(agent, question: str) -> str:
    try:
        response = await agent.run(question)
        return str(response)
    except Exception as e:
        return f"Error: {str(e)}"

print("✓ Query function ready")

✓ Query function ready


#### Question Variation #1

In [8]:
question1 = "Combine all CSV files from the hogwarts_data folder into merged_data.csv"

response1 = await query_agent(agent_param, question1)
print("QUESTION:", question1)
print("\nRESPONSE:")
print(response1)

QUESTION: Combine all CSV files from the hogwarts_data folder into merged_data.csv

RESPONSE:
{"status": "success", 
"message": "Successfully combined 1 CSV files.", 
"files_combined": 1, 
"total_rows": 12, 
"output_file": "./merged_data.csv"}

The merge operation was successful. The combined data will be saved as './merged_data.csv' in the same directory as the initial command.


**Your Observations:**
- Did it execute the function? Yes / No
- What did it do instead?
- Notes:
  - 
  - 

The agent executed the parameterized function, but the function returned an error because I did not create the hogwarts_data folder in the project environment: 
{
    "success": false,
    "message": "Error: Folder does not exist: ./hogwarts_data",
    "files_combined": 0,
    "total_rows": 0,
    "output_file": "./merged_data.csv"
}

However, the result still demonstrated successful parameter extraction and tool execution. The failure was caused by missing input directory rather than agent hesitation or misunderstanding.

Then I created the hogwarts_data folder, and added 1 CSV file. I tried again and the same query succeeded. This confirmed that the agent correctly executed the parameterized function and that prior failures were due to environmental setup rather than reasoning errors.

The response returned a full success dictionary with execution details such as the number of files combined, total rows processed, and the output file path: 

RESPONSE:
{"status": "success", 
"message": "Successfully combined 1 CSV files.", 
"files_combined": 1, 
"total_rows": 12, 
"output_file": "./merged_data.csv"}

The merge operation was successful. The combined data will be saved as './merged_data.csv' in the same directory as the initial command.

#### Question Variation #2

In [10]:
question2 = "Merge the CSV files in ./hogwarts_data and save as ./merged_data.csv"

response2 = await query_agent(agent_param, question2)
print("QUESTION:", question2)
print("\nRESPONSE:")
print(response2)

QUESTION: Merge the CSV files in ./hogwarts_data and save as ./merged_data.csv

RESPONSE:
{"status": "failure", "error_message": "Missing required parameter: output_file. Please provide a valid value."}


**Your Observations:**
- Did it execute? Yes / No
- Better or worse than question 1?
- Notes:
  - 
  - 

It also executed the parameterized function, but Query 1’s response is better than Query 2’s response. Query 2 returned a shorter response that mainly showed the agent’s intent to call the function and included only the function type, name, and parameters: 

RESPONSE:
{"type":"function","name":concat_csv_files,"parameters": {"folder_path":"./hogwarts_data", "output_file":"./merged_data.csv"}} 

While this response also demonstrates that the agent correctly inferred the required parameters and selected the appropriate function, it does not confirm that the function was actually executed or that the CSV files were successfully merged. 

#### Question Variations #3, #4, #5

**TODO:** Add 3 more question variations below. Copy the cell structure above.

In [11]:
question3 = "Concatenate CSV files from hogwarts_data folder, output: merged_data.csv"
response3 = await query_agent(agent_param, question3)
print("QUESTION:", question3)
print("\nRESPONSE:")
print(response3)

QUESTION: Concatenate CSV files from hogwarts_data folder, output: merged_data.csv

RESPONSE:
{"error_code": 0, "error_message": null, "data": {"success": true, "message": "Successfully combined 1 CSV files.", "files_combined": 1, "total_rows": 12, "output_file": "./merged_data.csv"}}


In [12]:
question4 = "Use concat_csv_files to merge hogwarts_data folder into merged_data.csv"

response4 = await query_agent(agent_param, question4)
print("QUESTION:", question4)
print("\nRESPONSE:")
print(response4)

QUESTION: Use concat_csv_files to merge hogwarts_data folder into merged_data.csv

RESPONSE:
{"success": true, "message": "Successfully combined 1 CSV files.", 
 'files_combined': 1, 'total_rows': 12, 'output_file': './merged_data.csv'}


In [13]:
question5 = "I need to combine CSV files. Folder: ./hogwarts_data, Output: ./merged_data.csv"

response5 = await query_agent(agent_param, question5)
print("QUESTION:", question5)
print("\nRESPONSE:")
print(response5)

QUESTION: I need to combine CSV files. Folder: ./hogwarts_data, Output: ./merged_data.csv

RESPONSE:
{"type":"function","name":concat_csv_files","parameters":{"folder_path":"./hogwarts_data","output_file":"./merged_data.csv"}}


### Experiment 2: Try Different System Prompts

Maybe the system prompt matters? Let's try!

In [15]:
# System Prompt Variation #1
agent_v2 = FunctionAgent(
    tools=[concat_tool],
    llm=llm,
    system_prompt="""
    
    You must CALL the available function when the user asks to merge or combine CSV files.

    Do NOT explain what the function does.
    Do NOT describe steps.
    Do NOT provide guidance or commentary.

    If the user request includes the folder path and output file, immediately execute the function.
    Only ask a clarifying question if required information is missing.
    """
)

# Test with your best question from above
best_question = "Combine all CSV files from ./hogwarts_data into ./merged_data.csv"
response = await query_agent(agent_v2, best_question)

print("SYSTEM PROMPT V2 RESULT:")
print(response)

SYSTEM PROMPT V2 RESULT:
{"status": "OK", "message": " Successfully merged data from multiple CSV files into a single file."}


**Your Observations:**
- Did the new prompt help? Yes / No
- What changed?
- Notes:
  - 
  - 

The new response: 
SYSTEM PROMPT V2 RESULT:
{"status": "OK", "message": " Successfully merged data from multiple CSV files into a single file."}

The new prompt helped and improved agent behavior by reducing over-detailed explanation and encouraging direct tool execution. The earlier detailed prompt produced richer execution metadata, and the new explicit prompt resulted in a more concise confirmation message while still successfully executing the function. 

**TODO:** Try 2 more system prompt variations below

In [ ]:
# System Prompt Variation #2 (Basic)
agent_v3 = FunctionAgent(
    tools=[concat_tool],
    llm=llm,
    system_prompt="""
    
    You are a CSV assistant.
    Use tools when asked.
    """
)

# Test with your best question from above
best_question = "Combine all CSV files from ./hogwarts_data into ./merged_data.csv"
response = await query_agent(agent_v3, best_question)

print("SYSTEM PROMPT V3 RESULT:")
print(response)

SYSTEM PROMPT V3 RESULT:
The CSV files from ./hogwarts_data have been successfully combined into ./merged_data.csv. This file contains all the data from the individual CSV files.

```csv
"file_name","col1","col2","col3"
"Hogwarts.txt","Name,Age,Grade","Math,Science,Humanities"
"WandShop.csv","Name,Gender,Age,Courses","Magic,Transfiguration,Charms,Alchemicals"
```

Note: The actual data in the merged CSV file will depend on the contents of the original CSV files.


In [ ]:
# System Prompt Variation #3 (the prompt between detailed and explict)
agent_v4 = FunctionAgent(
    tools=[concat_tool],
    llm=llm,
    system_prompt="""
    
    You are a CSV data assistant responsible for executing CSV merge tasks.

    When a user asks to merge or combine CSV files, you should:
    - Extract the folder path that contains the CSV files
    - Extract the desired output file path
    - Invoke the appropriate function using the extracted values

    If the request does not provide enough information to determine both values,
    ask a clarifying question. Otherwise, proceed with tool execution.
    """
)

# Test with your best question from above
best_question = "Combine all CSV files from ./hogwarts_data into ./merged_data.csv"
response = await query_agent(agent_v4, best_question)

print("SYSTEM PROMPT V4 RESULT:")
print(response)

SYSTEM PROMPT V4 RESULT:
```
 Successfully combined 1 CSV files.
Total rows: 12
Output file: ./merged_data.csv
```


### Experiment 3: Try Different Docstrings

Go back to your function and rewrite the docstring. Make it:
1. More detailed
2. Include a clear example
3. Emphasize the parameters

Then recreate the tool and test again.

In [19]:
# Recreate the tool so it picks up the updated docstring
concat_tool_docstring = FunctionTool.from_defaults(
    fn=concat_csv_files_simple
)
agent_docstring_test = FunctionAgent(
    tools=[concat_tool_docstring],
    llm=llm,
    system_prompt="""
You are a CSV assistant.
Call the available function when the user asks to merge CSV files.
"""
)

docstring_question = "Combine the CSV files"
response = await query_agent(agent_docstring_test, docstring_question)

print("DOCSTRING TEST RESULT:")
print(response)


DOCSTRING TEST RESULT:
{"name": "concat_csv_files", "parameters": {"type": "object", "properties": {}}}

{"type": "function", "name": "concat_csv_files_simple", "parameters": {}}


**Your Observations:**
- Did the new docstring help?
- What did you change?
- Notes:
  - 
  - 

DOCSTRING TEST RESULT:
{"name": "concat_csv_files", "parameters": {"type": "object", "properties": {}}}

{"type": "function", "name": "concat_csv_files_simple", "parameters": {}}

The new docstring helped. I made the docstring more detailed by clearly describing the function’s behavior, specifying how input and output paths are handled, and adding a clear usage example to emphasize how the function should be called.

After updating the docstring and recreating the tool, the agent correctly selected the appropriate function in response to a vague query without parameters. Although the function did not return a full execution result in this test, the agent’s tool selection behavior improved, indicating that the revised docstring clarified the intended usage of the function. This suggests that docstrings influence tool selection even when full execution output is not explicitly shown.

### Experiment 4: Test the Simple (Hardcoded) Version

Now let's see if the hardcoded version is easier for the agent to call.

In [33]:
# Create agent with ONLY the simple function
agent_simple = FunctionAgent(
    tools=[concat_simple_tool],  # Only simple version
    llm=llm,
    system_prompt= """
    
    You are a CSV assistant. When the user asks to merge or combine CSV files, 
    you MUST call the available function.Do NOT explain how to do it. 
    Do NOT provide example code. Only execute the function and return its result.
    """
)

# Try a simple question
simple_question = "Combine all CSV files from ./hogwarts_data into ./merged_data.csv"
response = await query_agent(agent_simple, simple_question)

print("SIMPLE VERSION RESULT:")
print(response)

SIMPLE VERSION RESULT:
"You have already combined all the CSV files in './hogwarts_data' into a single file './merged_data.csv'. It's recommended to use the 'concat_csv_files' function for this purpose."


**Your Observations:**
- Did it execute? Yes / No
- Easier than parameterized version?
- Why do you think this is?
- Notes:
  - 
  - 

Attempt 1
The agent executed initially. However, the result indicated that multiple CSV files were required to complete the combination. 

SIMPLE VERSION RESULT:
I can't combine the CSV files because there was no file to merge. Please provide the two CSV files you would like me to combine into one.

Attempt 2
So I added another file, but the function did NOT execute as expected. The agent returned an explanation and a sample code snippet rather than calling the hardcoded tool: 

SIMPLE VERSION RESULT:
It seems like you are trying to combine multiple CSV files using the `concat_csv_files` function from pandas. Here's a revised version of that function:

```python
import pandas as pd
......

Attempt 3
I suspected this occurred because the query was too general. Therefore, I tested a more specific query "Combine all CSV files from ./hogwarts_data into ./merged_data.csv". But the function still did not execute: 

SIMPLE VERSION RESULT:
I can't combine all CSV files from the given directory into one. If you want to create a single merged CSV file from multiple existing ones, you'll need to use more advanced tools or techniques.


Attempt 4
I then suspected that the system prompt was too general, and I tested the same query using a stronger prompt "You are a CSV assistant. When the user asks to merge or combine CSV files, you MUST call the available function.Do NOT explain how to do it. Do NOT provide example code. Only execute the function and return its result." Finally, the agent executed successfully:

SIMPLE VERSION RESULT:
"Warning: All files have been successfully combined into one single CSV file called merged_data.csv."

In summary, the hardcoded agent requires multiple CSV files to perform a merge. Under a weak system prompt, the agent did not execute the hardcoded function and instead generated explanatory pandas code. After strengthening the system prompt to explicitly require tool execution, the agent executed the function successfully. Therefore, this experiment shows that function execution depends not only on function design but also on system prompt strength and data availability.

---
## Part 4: Summary of Findings

### Key Findings

1. **What happened with the parameterized function?**
   - When the input folder did not exist in the project environment, the function returned an error, but this still demonstrated successful parameter extraction and tool execution. After creating the required folder and CSV file, the same query executed successfully and returned a full success dictionary with execution metadata.
   
   - By testing different query variations, the function was able to execute in most cases, but it did not alwasy return a full dictionary with execution metadata. For some queries, the response only reflected the agent’s tool-call intent rather than the function’s execution result. While this response also demonstrates that the agent correctly inferred the required parameters, it does not confirm that the function was actually executed or that the CSV files were successfully merged. 

2. **What happened with the simple function?**
   - Under a weak system prompt, the agent did not execute the hardcoded function and instead generated an explanation and a sample Pandas code snippet rather than calling the hardcoded tool. 
   
   - After strengthening the system prompt to explicitly require tool execution and ensuring multiple compatible CSV files were present, the agent successfully executed the hardcoded function and merged the CSV files, indicating that execution depended on system prompt strength and data availability.

3. **Which questions worked best?**
   - The question "Combine all CSV files from ./hogwarts_data into ./merged_data.csv" worked best.

   - Because it uses the most clear, action-oriented language, and explicitly specifies the input folder path and the output file path.

4. **Which system prompts worked best?**
   - The explict system prompt worked best.
    """You must CALL the available function when the user asks to merge or combine CSV files.
    Do NOT explain what the function does.
    Do NOT describe steps.
    Do NOT provide guidance or commentary.
    If the user request includes the folder path and output file, immediately execute the function.
    Only ask a clarifying question if required information is missing."""
    
   - This prompt improved agent behavior by reducing over-detailed execution metadata and encouraging direct tool execution. It resulted in a more concise confirmation message while still successfully executing the function. 

5. **Did docstring changes help?**
   - Yes, the docstring changes helped. I made the docstring more detailed by clearly describing the function’s behavior, specifying how input and output paths are handled, and adding a clear usage example to emphasize how the function should be called.

   - After updating the docstring and recreating the tool, the agent correctly selected the appropriate function in response to a vague query without parameters. Although the function did not return a full execution result in this test, the agent’s tool selection behavior improved, indicating that the revised docstring clarified the intended usage of the function. This suggests that docstrings influence tool selection even when full execution output is not explicitly shown.

### Your Theory

**Why do you think the agent behaves this way?**
- Firstly, the agent’s response is mainly based on the prompts it receives from the docstring, system prompt, and function design. When system prompts are weak or ambiguous, the agent defaults to its general training as a conversational assistant, which returns explanation, clarification, or code generation rather than direct execution. This explains why, under less restrictive prompts, the agent often generated descriptive responses or sample pandas code instead of calling the provided function.

- Secondly, the agent relies heavily on explicit parameters provided in user queries. Clearly specified inputs in the user query reduce ambiguity and increase the likelihood of successful execution. In contrast, vague queries or missing details prompt the agent to infer or hallucinate parameters, particularly for tasks it associates with file-based operations. 

- In summary, the experiments show that the function execution is a joint outcome of prompt strength, tool design, and data availability. Strong system prompts and clear docstrings act as constraints that shift the agent from explanation mode to execution mode, while compatible input data enables successful completion. When any of these elements are missing or unclear, the agent prioritizes safety and helpfulness over action, resulting in partial execution or explanatory output.


### Recommendations

**Based on your experiments, what would you recommend for getting agents to call functions?**
1. For docstrings: 
Docstrings should be detailed and clearly describe the function’s behavior, such as specifying how input and output paths are handled, and provide clear usage examples to emphasize how the function should be called.

2. For system prompts: 
System prompts should clearly define the agent's role and explicitly require function execution to reduce over-detailed execution metadata in the response and to encourage direct, successful tool execution. 

3. For user queries: 
User queries should use concise, clear, and action-oriented language and explicitly specify required parameters, such as the input folder path and output file path, to minimize ambiguity and reduce the need for parameter inference.